# Part 3: উন্নত রিমোট এক্সিকিউশন সরঞ্জাম (Advanced Remote Execution Tools)

শেষ বিভাগে আমরা ফেডারেটেড লার্নিং ব্যবহার করে একটি খেলনা মডেল প্রশিক্ষণ দিয়েছি। আমরা এটি করেছি আমাদের মডেলের উপর .send() এবং .get() কল করে, এটিকে প্রশিক্ষণ ডেটার অবস্থানটিতে প্রেরণ করে, এটি আপডেট করে এবং তারপরে ফিরিয়ে আনা। তবে উদাহরণটির শেষে আমরা বুঝতে পেরেছিলাম যে মানুষের গোপনীয়তা রক্ষা করতে আমাদের আরও কিছুটা এগিয়ে যাওয়া দরকার। যথা- আমরা .get() কল করার **পূর্বে** গ্রেডিয়েন্টগুলি গড় করতে চাই। এইভাবে, আমরা কখনই কারোর সঠিক গ্রেডিয়েন্টটি দেখতে পাবো না (এভাবে তাদের গোপনীয়তা রক্ষা করা আরও ভাল !!!

তবে এটি করার জন্য আমাদের আরও কয়েকটি টুকরো দরকার:

- অন্য কোনও শ্রমিককে সরাসরি টেন্সর প্রেরণের জন্য একটি পয়েন্টার ব্যবহার করুন

এছাড়াও, আমরা এখানে থাকাকালীন আমরা আরও কয়েকটি উন্নত টেনসর অপারেশন সম্পর্কে শিখতে যা এই উদাহরণ এবং ভবিষ্যতে আমাদের দু'জনকেই সহায়তা করবে!

Authors:

- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

অনুবাদক:

- Sourav Das - Twitter: [@adventuroussrv](https://twitter.com/adventuroussrv)

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

# বিভাগ ৩.১ - পয়েন্টারগুলিতে পয়েন্টার (Section 3.1 - Pointers to Pointers)

আপনি যেমন জানেন, পয়েন্টারটেন্সর অবজেক্টগুলি সাধারণ টেনসরের মতোই অনুভূত হয়। প্রকৃতপক্ষে, তারা অনেক বেশি টেনসরের মতো যার জন্য আমরা পয়েন্টারগুলি **থেকেও** পয়েন্টার রাখতে পারি। এটা দেখো

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [ ]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

In [ ]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

In [ ]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

### কি হলো? (What happened?)

সুতরাং, পূর্ববর্তী উদাহরণে আমরা `x` নামক একটি টেন্সর তৈরি করেছি  এবং এটি Bob-কে প্রেরণ করি। আমাদের স্থানীয় মেশিনে একটি পয়েন্টার তৈরি করি (`x_ptr`)

তারপরে, আমরা `x_ptr.send(alice)`-কে কল করি,  যেটি Alice কে **পয়েন্টারটি পাঠিয়ে দিয়েছে**

বিঃদ্রঃ এটি তথ্য সরায়নি! পরিবর্তে, এটি ডেটাতে পয়েন্টার সরিয়েছে 

In [ ]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

In [ ]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

In [ ]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

In [ ]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

### পয়েন্টারে গাণিতিক -> পয়েন্টার -> ডেটা অবজেক্ট (Arithmetic on Pointer -> Pointer -> Data Object)

এবং ঠিক সাধারণ পয়েন্টারগুলির মতো, আমরা এই টেন্সরগুলি জুড়ে ইচ্ছামত পাইটর্চ অপারেশন করতে পারি

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

# বিভাগ 3.2 - পয়েন্টার চেইন অপারেশনস (Section 3.2 - Pointer Chain Operations)

সুতরাং, শেষ বিভাগে যখনই আমরা .send() অথবা .get() অপারেশনকে কল করেছি, এটি আমাদের স্থানীয় মেশিনে  সরাসরি টেনসরের সেই অপারেশনটিকে কল করতো। যাইহোক, যদি আপনার পয়েন্টারগুলির একটি শৃঙ্খল(chain) থাকে তবে কখনও কখনও আপনি চেইনের **শেষ** পয়েন্টারে .get() অথবা .send() জাতীয়  অপারেশনগুলিকে কল করতে চান(যেমন সরাসরি একজন শ্রমিকের কাছ থেকে অন্য শ্রমিকের কাছে ডেটা প্রেরণ). এটি সম্পাদন করার জন্য, আপনি বিশেষ করে এই গোপনীয়তা সংরক্ষণের জন্য পরিচালিত ফাংশনগুলি ব্যবহার করতে চান।

এই অপারেশনগুলি হলো:

- `my_pointer2pointer.move(another_worker)`

In [ ]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x = x.move(alice)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x

অসাধারণ! এখন আমরা একটি বিশ্বস্ত অ্যাগ্রিগেটর ব্যবহার করে রিমোট **gradient averaging** সঞ্চালনের জন্য প্রয়োজনীয় সরঞ্জামগুলি দিয়ে সজ্জিত।

# অভিনন্দন !!! - সম্প্রদায় যোগদানের সময়! (Congratulations!!! - Time to Join the Community!)

এই নোটবুক টিউটোরিয়ালটি সম্পন্ন করার জন্য অভিনন্দন! আপনি যদি এটি উপভোগ করেন এবং গোপনীয়তা সংরক্ষণ, এআই এবং এআই সরবরাহ চেইনের (ডেটা) বিকেন্দ্রীভূত মালিকানার দিকে আন্দোলনে যোগ দিতে চান, আপনি নিম্নলিখিত উপায়ে এটি করতে পারেন

###  গিটহাবে পাইসিফ্ট কে স্টার দিন (Star PySyft on GitHub)

আমাদের সম্প্রদায়কে সাহায্য করার সবচেয়ে সহজ উপায় হলো রিপোসিটোরি গুলোতে ষ্টার করা। এটি আমরা যে অসাধারণ সরঞ্জামগুলি তৈরি করছি তার সচেতনতা বাড়াতে সহায়তা করে।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### আমাদের স্ল্যাক যোগ দিন (Join our Slack!)

সর্বশেষতম অগ্রগতিতে আপ টু ডেট রাখার সর্বোত্তম উপায় হল আমাদের সম্প্রদায়ে যোগদান করা! আপনি ফর্মটি পূরণ করে এটি করতে পারেন

[http://slack.openmined.org](http://slack.openmined.org)

### একটি কোড প্রকল্পে যোগদান করুন (Join a Code Project!)

আমাদের সম্প্রদায়ে অবদান রাখার সর্বোত্তম উপায় হলো কোড অবদানকারী হয়ে উঠুন। যে কোনও সময় আপনি পাইসাইফ্ট গিটহাবে ইস্যু পৃষ্ঠাতে যেতে পারেন এবং "প্রকল্পগুলি" এর জন্য ফিল্টার করতে পারেন। এটি আপনাকে শীর্ষ স্তরের সমস্ত টিকিট দেখিয়ে দেবে যে আপনি কোন প্রকল্পগুলিতে যোগদান করতে পারেন তার একটি ওভারভিউ দেয়! আপনি যদি কোনও প্রকল্পে যোগ দিতে না চান তবে আপনি কিছুটা কোডিং করতে চান তবে আপনি "ভাল প্রথম ইস্যু" চিহ্নিত গিটহাব ইস্যুগুলি অনুসন্ধান করে আরও "ওয়ান অফ" মিনি-প্রকল্পগুলির সন্ধান করতে পারেন।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### দান করা (Donate)

আপনার যদি আমাদের কোডবেসে অবদান রাখার সময় না থাকে তবে তবুও সমর্থন দিতে চান, আপনি আমাদের ওপেন কালেক্টিভেরও Backer হয়ে উঠতে পারেন। সমস্ত অনুদান আমাদের ওয়েব হোস্টিং এবং অন্যান্য সম্প্রদায় ব্যয় যেমন হ্যাকাথনস এবং মেটআপগুলির (hackathons and meetups!) দিকে যায়
[OpenMined's Open Collective Page](https://opencollective.com/openmined)